# Single-cell RNASeq data analysis in python - 5 time points

## First, the needed libraries are loaded

In [ ]:
import numpy as nu

In [ ]:
# import scanpy as sc
import pandas as pd
from scipy.sparse import csr_matrix

from matplotlib import pyplot as plt
from matplotlib import rcParams
%matplotlib inline

In [ ]:
import scanpy as sc

In [ ]:
import leidenalg

In [ ]:
def clustering_score(original_adata, score_value = 'bic', clustering_algorithm='leiden', dim_reduction = 'pca', min_res=0.1, max_res=2.0, step=0.1, plot=True):
    #calinski_harabasz
    import numpy as nu
    import scanpy as sc
    import numpy as nu
    from sklearn.metrics import calinski_harabasz_score

    sc.settings.verbosity = 0

    if dim_reduction == 'pca':
        dim_reduction = 'X_pca'
    elif dim_reduction == 'umap':
        dim_reduction = 'X_umap'
    else:
        print('please choose pca or umap as dimensionality reduction')
        exit

    res = list(nu.arange(min_res,max_res,step))
    score = []
    n_clus = []

    for r in res:
        index = res.index(r)
        adata = original_adata.copy()
        string_r = str(r)
        clustering_name = '%s_res%s' %(clustering_algorithm,string_r)
        print('Clustering by using the resolution %.2f, step %i of %i' %(r,index+1,len(res)))
        if clustering_algorithm == 'leiden':
            sc.tl.leiden(adata, key_added="%s_res%s" %(clustering_algorithm,string_r), resolution=r)
        elif clustering_algorithm == 'louvain':
            sc.tl.louvain(adata, key_added="%s_res%s" %(clustering_algorithm,string_r), resolution=r)
        else:
            print('please choose louvain or leiden as clustering_algorithm')
            exit
        if score_value == 'bic':
            score_name = 'BIC score'
            n_points = len(adata.obs[clustering_name])
            n_clusters = len(set(adata.obs[clustering_name]))
            n_dimensions = adata.obsm[dim_reduction].shape[1]

            n_parameters = (n_clusters - 1) + (n_dimensions * n_clusters) + 1

            loglikelihood=0
            for cluster_id in set(adata.obs[clustering_name]):
                cluster_mask = (adata.obs[clustering_name] == cluster_id)
                X_cluster = adata.obsm[dim_reduction][cluster_mask]

                n_points_cluster = len(X_cluster)
                centroid = nu.mean(X_cluster, axis=0)
                variance = nu.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
                loglikelihood += \
                  n_points_cluster * nu.log(n_points_cluster) \
                  - n_points_cluster * nu.log(n_points) \
                  - n_points_cluster * n_dimensions / 2 * nu.log(2 * nu.pi * variance) \
                  - (n_points_cluster - 1) / 2

            bic = loglikelihood - (n_parameters / 2) * nu.log(n_points)
            bic = -2*bic
            score.append(bic)
            n_clus.append(n_clusters)

            del adata.obs[clustering_name]
        
        elif score_value == 'calinski':
            score_name = 'Calinski-Harabasz score'
            n_clusters = len(set(adata.obs[clustering_name]))
            n_clus.append(n_clusters)
            score.append(-1*calinski_harabasz_score(adata.obsm[dim_reduction], adata.obs[clustering_name]))
            

    df = pd.DataFrame()
    df[score_name] = score
    df['resolution'] = res
    df['n_clus'] = n_clus

    if plot==True:
        df.plot(x='resolution',y=score_name)
    print('\n')

    return df, res[nu.argmin(score)]

### The following parameters set the level of verbosity of the scanpy library and the resolution of the figures that will be produced. 

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=200, facecolor='white')

## Load the datasets

Below, insert the path of the 3 datasets

In [ ]:
adata_9396h_path = "C:/Users/cguiller/Documents/SERGIO_analysis/data_singlecell/filtered_feature_bc_matrix9396"
adata_108hAEL_path = "C:/Users/cguiller/Documents/SERGIO_analysis/data_singlecell/filtered_feature_bc_matrix_108AEL"
adata_WP_path = "C:/Users/cguiller/Documents/SERGIO_analysis/data_singlecell/filtered_feature_bc_matrix_WP"
adata_5h_path = "C:/Users/cguiller/Documents/SERGIO_analysis/data_singlecell/filtered_feature_bc_matrix_5hAPF"
adata_12h_path = "C:/Users/cguiller/Documents/SERGIO_analysis/data_singlecell/filtered_feature_bc_matrix_12hAPF"

The function sc.read_10x_mtx() from scanpy is equivalent to Read10X() function in Seurat, but it produces an AnnData object instead than a Seurat object. In fact, scanpy deals with AnnData objects

In [ ]:
adata_9396h = sc.read_10x_mtx(adata_9396h_path, var_names = 'gene_symbols', cache = True)
adata_108hAEL = sc.read_10x_mtx(adata_108hAEL_path, var_names = 'gene_symbols', cache = True)
adata_WP = sc.read_10x_mtx(adata_WP_path, var_names = 'gene_symbols', cache = True)
adata_5h = sc.read_10x_mtx(adata_5h_path, var_names = 'gene_symbols', cache = True)
adata_12h = sc.read_10x_mtx(adata_12h_path, var_names = 'gene_symbols', cache = True)

The function adata.var_names_make_unique() nakes the gene names unique by appending a number string to each duplicate index element: ‘1’, ‘2’, etc.

In [ ]:
adata_9396h.var_names_make_unique()
adata_108hAEL.var_names_make_unique()
adata_WP.var_names_make_unique()
adata_5h.var_names_make_unique()
adata_12h.var_names_make_unique()

## Preprocess the data and filter out genes and cells

N.B. What is done in the cell below is a purely technical thing, with no biological value.
Basically, we noticed that there is a gene called 'nan'. Python syntaxis recognize it as a NaN (not a number), giving problem for the downstream analysis. Let's remove it from the annDatas (without this step, there would be problems for the next step)

In [ ]:
indices_to_keep_9396h = nu.in1d(adata_9396h.var_names.values.astype(str), 'nan')
adata_9396h = adata_9396h[:,~indices_to_keep_9396h]

indices_to_keep_108hAEL = nu.in1d(adata_108hAEL.var_names.values.astype(str), 'nan')
adata_108hAEL = adata_108hAEL[:,~indices_to_keep_108hAEL]

indices_to_keep_WP = nu.in1d(adata_WP.var_names.values.astype(str), 'nan')
adata_WP = adata_WP[:,~indices_to_keep_WP]

indices_to_keep_5h = nu.in1d(adata_5h.var_names.values.astype(str), 'nan')
adata_5h = adata_5h[:,~indices_to_keep_5h]

indices_to_keep_12h = nu.in1d(adata_12h.var_names.values.astype(str), 'nan')
adata_12h = adata_12h[:,~indices_to_keep_12h]

Now, we add some metadata to the AnnData objects, in order to create one merged object later and keep memory of the original datasets.

In [ ]:
adata_9396h.obs['sample'] = '9396h'
adata_108hAEL.obs['sample'] = '108hAEL'
adata_WP.obs['sample'] = 'WP'
adata_5h.obs['sample'] = '5h'
adata_12h.obs['sample'] = '12h'

### Concatenate the 3 samples and delete individual datasets to save space
The 3 datasets are simply concatenated by using the function adata.concatetante(), that accepts as arguments the names of the AnnData objects containing the data. After concatenating all the datasets and one single AnnData object is created (adata), the single datasets are deleted, in order so save memory


In [ ]:
adata = adata_9396h.concatenate(adata_108hAEL, adata_WP, adata_5h, adata_12h)
del(adata_9396h, adata_108hAEL, adata_WP, adata_5h, adata_12h)

In [ ]:
adata

### Calculate some metrics for the merged dataset, for QC
Ribosomal genes and micochondrial genes are detected, and their percentage in each cell is generated and saved

In [ ]:
adata.var['ribo'] = adata.var_names.str.startswith(("RpS","RpL"))
adata.var['mt'] = adata.var_names.str.startswith(("Mt"))

sc.pp.calculate_qc_metrics(adata, qc_vars=['ribo', 'mt'], percent_top=None, log1p=False, inplace=True)

Check the most expressed genes of the dataset

In [ ]:
sc.settings.set_figure_params(dpi=100, facecolor='white')
sc.pl.highest_expr_genes(adata, n_top=20, )

### Filtering cells based on expressed mitochondrial genes and then for total number of counts

Let's visualize the mitochondrial genes in each sample before filtering the cells

In [ ]:
sc.settings.set_figure_params(dpi = 100, facecolor='white')
sc.pl.violin(adata, ['pct_counts_mt'], jitter=0.4, groupby = 'sample', rotation= 45)

Not really a big amount of mitochondrial genes, we could avoid any filtering. Just to keep track of standard procedures, below there is a line of code to filter the cells expressing more than 20% of mitochondrial genes

In [ ]:
adata = adata[adata.obs['pct_counts_mt'] < 20, :]

print("Remaining cells %d"%adata.n_obs)

Let's visualize the number of genes and the total counts for each sample, in order to decide what to filter

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'], jitter=0.4, groupby = 'sample', rotation= 45)

According to what we saw above, it would be reasonable to keep cells expressing more than 600 and less than 5000 genes, and a total number of counts lower than 150000. This can be done by using the following lines of code:

In [ ]:
# filter per nCounts  and nFeatures
adata = adata[adata.obs['n_genes_by_counts'] > 600, :]
adata = adata[adata.obs['n_genes_by_counts'] < 5000, :]
adata = adata[adata.obs['total_counts'] < 150000, :]

Let's now visualize number of genes and total counts per sample, after the filtering:

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'], jitter=0.4, groupby = 'sample', rotation= 45)

N.B. A very standard filter on cells, is to exclude cells expressing less than 200 genes. This can be done in as follow. In our case, we take this line of code to keep track of standard data analysis procedure, but it will be not relevant and it won't filter any additional cell.

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)

### Filter on gene that are expressed in less than 3 cells:

In [ ]:
sc.pp.filter_genes(adata, min_cells=3)

print(adata.n_obs, adata.n_vars)

In [ ]:
adata2 = adata

In [ ]:
adata2

### Custom filtering
In this particular analysis, we are interested in cells that do express at least one among the genes GFP, Mef2 and twi regarding the timesteps WP and 5h, and  we are interested in cells that do express at least one among the genes GFP and twi regarding the timesteps 9396.

In [ ]:
d9396_indices = adata.obs.loc[adata.obs['sample'] == '9396h'].index
adata_9396h = adata[d9396_indices]
GFP_indices = adata_9396h[(adata_9396h[: , 'GFP'].X > 0) ].obs.index
twi_indices = adata_9396h[(adata_9396h[: , 'twi'].X > 0) ].obs.index
indices_9396 = [*set([i for i in GFP_indices] + [i for i in twi_indices])]

In [ ]:
d108hAEL_indices = adata.obs.loc[adata.obs['sample'] == '108hAEL'].index
adata_108hAEL = adata[d108hAEL_indices]
GFP_indices = adata_108hAEL[(adata_108hAEL[: , 'GFP'].X > 0) ].obs.index
twi_indices = adata_108hAEL[(adata_108hAEL[: , 'twi'].X > 0) ].obs.index
indices_108hAEL = [*set([i for i in GFP_indices] + [i for i in twi_indices])]

In [ ]:
dWP_indices = adata.obs.loc[adata.obs['sample'] == 'WP'].index
adata_WP = adata[dWP_indices]
GFP_indices = adata_WP[(adata_WP[: , 'GFP'].X > 0) ].obs.index
twi_indices = adata_WP[(adata_WP[: , 'twi'].X > 0) ].obs.index
Mef2_indices = adata_WP[(adata_WP[: , 'Mef2'].X > 0) ].obs.index
indices_WP = [*set([i for i in GFP_indices] + [i for i in twi_indices] + [i for i in Mef2_indices])]

In [ ]:
d5h_indices = adata.obs.loc[adata.obs['sample'] == '5h'].index
adata_5h = adata[d5h_indices]
GFP_indices = adata_5h[(adata_5h[: , 'GFP'].X > 0) ].obs.index
twi_indices = adata_5h[(adata_5h[: , 'twi'].X > 0) ].obs.index
Mef2_indices = adata_5h[(adata_5h[: , 'Mef2'].X > 0) ].obs.index
indices_5h = [*set([i for i in GFP_indices] + [i for i in twi_indices] + [i for i in Mef2_indices])]

In [ ]:
d12h_indices = adata.obs.loc[adata.obs['sample'] == '12h'].index
adata_12h = adata[d12h_indices]
GFP_indices = adata_12h[(adata_12h[: , 'GFP'].X > 0) ].obs.index
twi_indices = adata_12h[(adata_12h[: , 'twi'].X > 0) ].obs.index
Mef2_indices = adata_12h[(adata_12h[: , 'Mef2'].X > 0) ].obs.index
indices_12h = [*set([i for i in GFP_indices] + [i for i in twi_indices] + [i for i in Mef2_indices])]

In [ ]:
kept_indices = [*set([i for i in indices_9396] + [i for i in indices_108hAEL] + [i for i in indices_WP] + [i for i in indices_5h] + [i for i in indices_12h])]

In [ ]:
adata = adata[kept_indices]

After the filterings, let's visualize some metrics on the entire dataset:

In [ ]:
sc.settings.set_figure_params(dpi = 300, facecolor='white')
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.settings.set_figure_params(dpi = 100, facecolor='white')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

## Data normalization and scaling
Once the data have been preprocessed, and cells and genes have been filtered out, we can proceed with normalization and scaling

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

### Identify high variable genes

In [ ]:
sc.pp.highly_variable_genes(adata, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(adata)


Before scaling the data, let's save somewhere the unscaled data, for eventual future use

In [ ]:
adata.raw = adata

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.

In [ ]:
import numpy.core.multiarray

sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

Scale the data

In [ ]:
sc.pp.scale(adata, max_value=10)

## Principal Component Analysis

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sc.settings.set_figure_params(dpi = 100, facecolor='white')
sc.pl.pca(adata, color='sample')

## Computing Neighborhood Graph

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

## Calculating UMAP

In [ ]:
adata = sc.read_h5ad("C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_before_removing_batch_effect_with_last_stage.h5ad")

In [ ]:
adata

In [ ]:
sc.tl.umap(adata)

sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.umap(adata, color='sample', title = "Pre batch correction")

In [ ]:
results_file = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_before_removing_batch_effect_with_last_stage.h5ad"
adata.write(results_file)

## Integrate datasets

The reference for the following part is here:
https://nbisweden.github.io/workshop-scRNAseq/labs/compiled/scanpy/scanpy_03_integration.html.
We will use BBKNN method to integrate the data

As the stored AnnData object contains scaled data based on variable genes, we need to make a new object with the logtransformed normalized counts. The new variable gene selection should not be performed on the scaled data matrix.

In [ ]:
adata2 = adata.raw.to_adata() 
adata2.uns['log1p']['base']=None

### Detect variable genes

If variable genes are selected across all the datasets, there is th risk of obtaining batch-specific genes which will drive the rest of the analysis. 
Then, we select variable genes from each batch separately

In [ ]:
sc.pp.highly_variable_genes(adata2, batch_key = 'sample')

Finally, we select the genes that are variable in at least 2 datasets and use them for remaining analysis.

In [ ]:
var_select = adata2.var.highly_variable_nbatches > 2
var_genes = var_select.index[var_select]
len(var_genes)

### BBKNN

In [ ]:
adata2 = adata.raw.to_adata() 


sc.external.pp.bbknn(adata2, batch_key='sample', n_pcs=30)  # running bbknn 1.3.6
sc.tl.umap(adata2)

sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.umap(adata2, color="sample", title="BBKNN Corrected umap")

Save the data adjusted with BBKNN

In [ ]:
saving_file = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_with_last_stage_adjusted_with_bbknn.h5ad"
adata2.write(saving_file)

# Downstream analysis choosing BBKNN adjusted counts

In [ ]:
adjusted_adata = sc.read_h5ad("C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_with_last_stage_adjusted_with_bbknn.h5ad")

### Batch aware feature selection
We can perform batch-aware highly variable gene selection by setting the batch_key argument in the scanpy highly_variable_genes() function. scanpy will then calculate HVGs for each batch separately and combine the results by selecting those genes that are highly variable in the highest number of batches.
(https://www.sc-best-practices.org/cellular_structure/integration.html)

In [ ]:
sc.pp.highly_variable_genes(adjusted_adata, n_top_genes=3000, flavor="cell_ranger", batch_key='sample')


In [ ]:
##select only the high variable genes
HVG = [i for i in adjusted_adata.var.loc[adjusted_adata.var['highly_variable'] == True].index]

In [ ]:
#select genes that are highly variable in every sample
HVG_shared = [i for i in adjusted_adata.var.loc[HVG].loc[adjusted_adata.var.loc[HVG]['highly_variable_intersection'] == True].index]

In [ ]:
n_batches = adjusted_adata.var["highly_variable_nbatches"].value_counts()
ax = n_batches.plot(kind="bar")
n_batches

### Clustering
Make a copy of the annData file, and on the copy perform the clustering by choosing several different resolution parameters. 


In [ ]:
import leidenalg

In [ ]:
adata = adjusted_adata.copy()
res = list(nu.arange(0.1,1.6,0.1))
for r in res:
    string_r = str(r)
    sc.tl.leiden(adata, key_added="leiden_res%s" %string_r, resolution=r)

In [ ]:
col = []
for i in range(0,len(res)):
    col.append('leiden_res%s' %str(res[i]))

sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.umap(
    adata,
    color=col,
    legend_loc="on data",
)

In [ ]:
a = clustering_score(adjusted_adata, score_value = 'bic', min_res=0.1,max_res=2.0,step=0.1, plot=True, dim_reduction = 'pca') 

### Analysing with best resolution (1.2)
Check for the best clustering by using the BIC criterion:
https://towardsdatascience.com/are-you-still-using-the-elbow-method-5d271b3063bd

NOTE: A FUNCTION TO AUTOMATIZE THE CHOICE OF THE RESOLUTION PARAMETER HAS BEEN WRITTEN, AND IT GIVES 1.2 AS a local minumum

In [ ]:
adjusted_adata = sc.read("C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_adjusted_with_bbknn_with_last_stage_after_clustering.h5ad")
adata = adjusted_adata.copy()

In [ ]:
optimum_resolution = 1.2

Once the optimum resolution is found, we come back to the adjusted_adata, clustering by using the optimum parameter

In [ ]:
sc.tl.leiden(adjusted_adata, key_added="leiden_res%s" %optimum_resolution, resolution=optimum_resolution)

Visualize the clustering

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')

sc.pl.umap(
    adjusted_adata,
    color=["leiden_res%s" %optimum_resolution],
    size=20, 
    legend_loc="on data",
)

Visualize only the cells belonging from each of the datasets separately

In [ ]:
d9396_indices = adjusted_adata.obs.loc[adjusted_adata.obs['sample'] == '9396h'].index
adjusted_adata_9396h = adjusted_adata[d9396_indices]

d108hAEL_indices = adjusted_adata.obs.loc[adjusted_adata.obs['sample'] == '108hAEL'].index
adjusted_adata_108hAEL = adjusted_adata[d108hAEL_indices]

dWP_indices = adjusted_adata.obs.loc[adjusted_adata.obs['sample'] == 'WP'].index
adjusted_adata_WP = adjusted_adata[dWP_indices]

d5_indices = adjusted_adata.obs.loc[adjusted_adata.obs['sample'] == '5h'].index
adjusted_adata_5h = adjusted_adata[d5_indices]

d12_indices = adjusted_adata.obs.loc[adjusted_adata.obs['sample'] == '12h'].index
adjusted_adata_12h = adjusted_adata[d12_indices]

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.umap(adjusted_adata[adjusted_adata.obs["sample"] == '9396h'], color=["leiden_res%s" %optimum_resolution], size=20, legend_loc="on data", title='9396h', show=False)

Visualize only the cells belonging from each of the datasets separately

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')

new_labels = ["93h AEL","108h AEL", "0h APF","5h APF", "12h APF"]

fig, ax = plt.subplots()
sc.pl.umap(
    adjusted_adata,
    color = "sample",
    title = '',
    show = False,
    size = 20,
    ax = ax,
    palette = {"9396h": "#440154FF", "108hAEL": "#414487FF", "WP": "#2A788EFF", "5h": "#22A884FF", "12h": "#7AD151FF"},
)
handles, labels = ax.get_legend_handles_labels()
order = [3,2,4,0,1]
ax.legend([handles[idx] for idx in order], new_labels, loc='center left', bbox_to_anchor=(1, 0.5), frameon=False)


In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.umap(
    adjusted_adata,
    color ="sample",
    title = '93hAEL',
    groups = '9396h',
    size=20, 
    show=False
)


In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.umap(
    adjusted_adata,
    color = "sample",
        title = '108hAEL',
    groups = '108hAEL',
    size=20, 
    show=False
)

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.umap(
    adjusted_adata,
    color = "sample",
        title = '0hAPF',
    groups = 'WP',
    size=20, 
    show=False
)

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.umap(
    adjusted_adata,
    color = "sample",
        title = '5hAPF',
    groups = '5h',
    size=20, 
    show=False
)

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.umap(
    adjusted_adata,
    color = "sample",
        title = '12hAPF',
    groups = '12h',
    size=20, 
    show=False
)

Check the clustering by visualizing only the cells from each of the dataset separately

In [ ]:
resulting_file = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_adjusted_with_bbknn_with_last_stage_after_clustering.h5ad"
adjusted_adata.write(resulting_file)

### Marker genes
Find the marker genes 

In [ ]:
adjusted_adata = sc.read("C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_adjusted_with_bbknn_with_last_stage_after_clustering.h5ad")
adata = adjusted_adata.copy()
adata.uns['log1p']['base']=None

In [ ]:
optimum_resolution=1.2
sc.tl.rank_genes_groups(adata, "leiden_res%s" %optimum_resolution, method='t-test')

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
##choose the number top marker genes that you want to visualize
n_genes = 20

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names'])[0:n_genes]

In [ ]:
filename = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/marker_genes_t_test_resolution_%s.tsv" %optimum_resolution
pd.DataFrame(adata.uns['rank_genes_groups']['names']).to_csv(filename, sep = '\t', index = None)

In [ ]:
resulting_file = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_with_last_stage_adjusted_with_bbknn_after_clustering_and_with_marker_genes.h5ad"
adata.write(resulting_file)

### In the following part, I want to visualize the markers of the clusters but obtaining them only the cells of the same temporal stage.

#### Timestep 9396h

In [ ]:
dict_ncells = dict({})
for i in nu.unique(adjusted_adata_9396h.obs['leiden_res1.2']):
    dict_ncells.update({"cluster_"+i: adjusted_adata_9396h[adjusted_adata_9396h.obs['leiden_res1.2'] == i].n_obs})
print(dict_ncells)


In [ ]:
adata2 = adjusted_adata_9396h
adata2.uns['log1p']['base']=None
sc.tl.rank_genes_groups(adjusted_adata_9396h, "leiden_res%s" %optimum_resolution, method='t-test')

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.rank_genes_groups(adjusted_adata_9396h, n_genes=25, sharey=False)

Visualize and save the marker genes

In [ ]:
##choose the number top marker genes that you want to visualize
n_genes = 20

In [ ]:
pd.DataFrame(adjusted_adata_9396h.uns['rank_genes_groups']['names'])[0:n_genes]

In [ ]:
filename = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/marker_genes_t_test_resolution_%s_only_9396h.tsv" %optimum_resolution
pd.DataFrame(adjusted_adata_9396h.uns['rank_genes_groups']['names']).to_csv(filename, sep = '\t', index = None)

#### Timestep 108hAEL

In [ ]:
dict_ncells = dict({})
for i in nu.unique(adjusted_adata_108hAEL.obs['leiden_res1.2']):
    dict_ncells.update({"cluster_"+i: adjusted_adata_108hAEL[adjusted_adata_108hAEL.obs['leiden_res1.2'] == i].n_obs})
print(dict_ncells)

In [ ]:
#exclude the clusters  because they only have one cell in this time step
adjusted_adata_108hAEL = adjusted_adata_108hAEL[adjusted_adata_108hAEL.obs['leiden_res1.2'] != '10']


In [ ]:
adata2 = adjusted_adata_108hAEL
adata2.uns['log1p']['base']=None
sc.tl.rank_genes_groups(adjusted_adata_108hAEL, "leiden_res%s" %optimum_resolution, method='t-test')


In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.rank_genes_groups(adjusted_adata_108hAEL, n_genes=25, sharey=False)

In [ ]:
##choose the number top marker genes that you want to visualize
n_genes = 20

In [ ]:
pd.DataFrame(adjusted_adata_108hAEL.uns['rank_genes_groups']['names'])[0:n_genes]

In [ ]:
filename = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/marker_genes_t_test_resolution_%s_only_108hAEL.tsv" %optimum_resolution
pd.DataFrame(adjusted_adata_108hAEL.uns['rank_genes_groups']['names']).to_csv(filename, sep = '\t', index = None)

#### Timestep WP

In [ ]:
dict_ncells = dict({})
for i in nu.unique(adjusted_adata_WP.obs['leiden_res1.2']):
    dict_ncells.update({"cluster_"+i: adjusted_adata_WP[adjusted_adata_WP.obs['leiden_res1.2'] == i].n_obs})
print(dict_ncells)

In [ ]:
adata2 = adjusted_adata_WP
adata2.uns['log1p']['base']=None
sc.tl.rank_genes_groups(adjusted_adata_WP, "leiden_res%s" %optimum_resolution, method='t-test')

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.rank_genes_groups(adjusted_adata_WP, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adjusted_adata_WP.uns['rank_genes_groups']['names'])[0:n_genes]

In [ ]:
filename = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/marker_genes_t_test_resolution_%s_only_WP.tsv" %optimum_resolution
pd.DataFrame(adjusted_adata_WP.uns['rank_genes_groups']['names']).to_csv(filename, sep = '\t', index = None)

#### Timestep 5h

In [ ]:
dict_ncells = dict({})
for i in nu.unique(adjusted_adata_5h.obs['leiden_res1.2']):
    dict_ncells.update({"cluster_"+i: adjusted_adata_5h[adjusted_adata_5h.obs['leiden_res1.2'] == i].n_obs})
print(dict_ncells)

In [ ]:
adata2 = adjusted_adata_5h
adata2.uns['log1p']['base']=None
sc.tl.rank_genes_groups(adjusted_adata_5h, "leiden_res%s" %optimum_resolution, method='t-test')

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.rank_genes_groups(adjusted_adata_5h, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adjusted_adata_5h.uns['rank_genes_groups']['names'])[0:n_genes]

In [ ]:
filename = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/marker_genes_t_test_resolution_%s_only_5h.tsv" %optimum_resolution
pd.DataFrame(adjusted_adata_5h.uns['rank_genes_groups']['names']).to_csv(filename, sep = '\t', index = None)

#### Timestep 12h APF

In [ ]:
dict_ncells = dict({})
for i in nu.unique(adjusted_adata_12h.obs['leiden_res1.2']):
    dict_ncells.update({"cluster_"+i: adjusted_adata_12h[adjusted_adata_12h.obs['leiden_res1.2'] == i].n_obs})
print(dict_ncells)


In [ ]:
adata2 = adjusted_adata_12h
adata2.uns['log1p']['base']=None
sc.tl.rank_genes_groups(adjusted_adata_12h, "leiden_res%s" %optimum_resolution, method='t-test')

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.rank_genes_groups(adjusted_adata_12h, n_genes=25, sharey=False)


In [ ]:
pd.DataFrame(adjusted_adata_12h.uns['rank_genes_groups']['names'])[0:n_genes]

In [ ]:
filename = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/marker_genes_t_test_resolution_%s_only_12h.tsv" %optimum_resolution
pd.DataFrame(adjusted_adata_12h.uns['rank_genes_groups']['names']).to_csv(filename, sep = '\t', index = None)

## Checking genes expression

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.umap(adjusted_adata, color='sv', legend_loc="on data", title='sv expression', show=False,size=20)

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
fig, axs = plt.subplots(2,3, figsize=(15,8),constrained_layout=True)
sc.pl.umap(adjusted_adata[adjusted_adata.obs["sample"] == '9396h'], color='tnc', legend_loc="on data", title='tnc gene in 9396h', ax=axs[0,0], show=False)
sc.pl.umap(adjusted_adata[adjusted_adata.obs["sample"] == '108hAEL'], color='tnc', legend_loc="on data", title='tnc gene in 108hAEL', ax=axs[0,1], show=False)
sc.pl.umap(adjusted_adata[adjusted_adata.obs["sample"] == 'WP'], color='tnc', legend_loc="on data", title='tnc gene in WP', ax=axs[0,2], show=False)
sc.pl.umap(adjusted_adata[adjusted_adata.obs["sample"] == '5h'], color='tnc', legend_loc="on data", title='tnc gene in 5h', ax=axs[1,0], show=False)
sc.pl.umap(adjusted_adata[adjusted_adata.obs["sample"] == '12h'], color='tnc', legend_loc="on data", title='tnc gene in 12h', ax=axs[1,1], show=False)

### Lineage tracing with PAGA

In [ ]:
adjusted_adata = sc.read_h5ad("C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_with_last_stage_adjusted_with_bbknn_after_clustering_and_with_marker_genes.h5ad")
optimum_resolution = 1.2

In [ ]:
adata = adjusted_adata

In [ ]:
adata.obs

In [ ]:
sc.tl.paga(adata, groups='leiden_res%s' %optimum_resolution)
sc.pl.paga(adata, color=['leiden_res%s' %optimum_resolution])

In [ ]:
sc.tl.draw_graph(adata, init_pos='paga', layout="drl")
sc.pl.draw_graph(adata, color=['leiden_res%s' %optimum_resolution], legend_loc='on data')

In [ ]:

sc.tl.draw_graph(adata, init_pos='paga', layout="drl")
sc.pl.draw_graph(adata, color=['sample'], legend_loc='on data')


In [ ]:
palette={"9396h": "#440154FF", "108hAEL": "#414487FF", "WP": "#2A788EFF", "5h": "#22A884FF", "12h": "#7AD151FF"}

In the following line, you choose a cluster that is supposed to choose a root: you suppose to follow the cell lineage starting from this cluster

In [ ]:
nu.flatnonzero(adata.obs['leiden_res%s' %optimum_resolution]=='11')[1]

In [ ]:
new_labels = ["5h APF", "12h APF", "108h AEL", "93h AEL","0h APF"]
sc.tl.dpt(adata)
sc.pl.draw_graph(adata, color=['sample'], title='PAGA analysis', palette={"9396h": "#440154FF", "108hAEL": "#414487FF", "WP": "#2A788EFF", "5h": "#22A884FF", "12h": "#7AD151FF"})

handles, labels = ax.get_legend_handles_labels()
order = [3,2,4,0,1]
ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order], new_labels, loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)


In [ ]:
palette={"9396h": "#440154FF", "108hAEL": "#414487FF", "WP": "#2A788EFF", "5h": "#22A884FF", "12h": "#7AD151FF" }
adata.uns['iroot'] = nu.flatnonzero(adata.obs['leiden_res%s' %optimum_resolution]=='11')[0]
sc.tl.dpt(adata)
sc.pl.draw_graph(adata, color=['sample', 'dpt_pseudotime'], legend_loc='on data')


In [ ]:
resulting_file = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_with_last_stage_adjusted_with_bbknn_after_clustering_and_paga_pseudotime.h5ad"
adata.write(resulting_file)

In [ ]:
adata = sc.read_h5ad("C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/adata_with_last_stage_adjusted_with_bbknn_after_clustering_and_paga_pseudotime.h5ad")

In [ ]:
sc.pl.draw_graph(adata, color='tnc', legend_loc='on data')

### PAGA on central cluster (4, 6, 11, 13, 14, 15)

In [ ]:
adata_central = adata[~adata.obs["leiden_res1.2"].isin(['0','1','2','3','5','7','8','9','10','12','16','17','18','19','20','21'])]

In [ ]:
sc.settings.set_figure_params(dpi = 200, facecolor='white')
fig, ax = plt.subplots()
sc.pl.umap(
    adata_central,
    color = "sample",
    title = '',
    show = False,
    size = 20,
    ax = ax,
    palette = {"9396h": "#440154FF", "108hAEL": "#414487FF", "WP": "#2A788EFF", "5h": "#22A884FF", "12h": "#7AD151FF"},
)
handles, labels = ax.get_legend_handles_labels()
order = [3,4,0,1,2]
ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order], loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)

In [ ]:
optimum_resolution=1.2
sc.tl.rank_genes_groups(adata_central, "leiden_res%s" %optimum_resolution, method='t-test')

sc.settings.set_figure_params(dpi = 200, facecolor='white')
sc.pl.rank_genes_groups(adata_central, n_genes=25, sharey=False)

n_genes = 20
pd.DataFrame(adata_central.uns['rank_genes_groups']['names'])[0:n_genes]

In [ ]:
filename = "C:/Users/cguiller/Documents/SERGIO_analysis/results/with108hAEL/marker_genes_t_test_resolution_%s_Clusster_central_top20.tsv" %optimum_resolution
pd.DataFrame(adata_central.uns['rank_genes_groups']['names'])[0:n_genes].to_csv(filename, sep = '\t', index = None)

In [ ]:
sc.tl.paga(adata_central, groups='leiden_res%s' %optimum_resolution)
sc.pl.paga(adata_central, color=['leiden_res%s' %optimum_resolution])

In [ ]:
sc.tl.draw_graph(adata_central, init_pos='paga', layout="drl")
sc.pl.draw_graph(adata_central, color=['leiden_res%s' %optimum_resolution], legend_loc='on data')

In [ ]:
sc.tl.dpt(adata_central)
sc.pl.draw_graph(adata_central, color=['sample'], title='PAGA analysis', palette={"9396h": "#440154FF", "108hAEL": "#414487FF", "WP": "#2A788EFF", "5h": "#22A884FF", "12h": "#7AD151FF"})


In [ ]:
adata_central.uns['iroot'] = nu.flatnonzero(adata_central.obs['leiden_res%s' %optimum_resolution]=='11')[0]
sc.tl.dpt(adata_central)
sc.pl.draw_graph(adata_central, color=['leiden_res%s' %optimum_resolution, 'dpt_pseudotime'],color_map='inferno',legend_loc='on data')
